### Create file name
### Get data

### Hazards of interest
### Air quality: Dust, Smoke, Fire
- ABI-L2-ADPC - Advanced Baseline Imager Level 2 Aerosol Detection CONUS
- ABI-L2-AODC - Advanced Baseline Imager Level 2 Aerosol Optical Depth CONUS
- ABI-L2-FDCC - Advanced Baseline Imager Level 2 Fire (Hot Spot Characterization) CONUS

### Lightning
- GLM-L2-LCFA - Geostationary Lightning Mapper Level 2 Lightning Detection

In [1]:
import datetime
from datetime import datetime
from pathlib import Path
import s3fs
import xarray as xr
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
from cartopy import crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from PIL import Image

In [2]:
def current_date():
    current_date = datetime.utcnow()
    year = str(current_date.strftime('%Y'))
    julian = str(current_date.strftime('%j')).zfill(3)
    # prev_hour = str(int(current_date.strftime('%H')) - 1).zfill(2)
    hour = str(current_date.strftime('%H')).zfill(2)
    # minutes = str(current_date.strftime('%M')).zfill(2)
    print(year, julian, hour)
    return year, julian, hour

In [3]:
def open_file_xarray(file):
    remote_file = fs.open(file, mode = 'rb')
    return xr.open_dataset(remote_file, engine = 'h5netcdf')

In [4]:
# Process ABI ADP smoke/dust detection (baseline algorithm)
def process_abi_adp_detection(ds):

    # Convert xarray Data Arrays to NumPy masked arrays w/correct dtype
    # Select "smoke present" (Smoke = 1) and "dust present" (Dust = 1) pixels
    smoke_detection = ds.Smoke.where(ds.Smoke == 1).to_masked_array().astype('uint8')
    dust_detection = ds.Dust.where(ds.Dust == 1).to_masked_array().astype('uint8')
    dqf = ds.DQF.to_masked_array().astype('uint16')

    # Process "Smoke" and "Dust" pixels using "DQF" variable flags
    # Select dust pixels outside of sun-glint areas using "DQF" bit 6
    # Flag values: outside sun-glint = 0, within sun-glint = 64
    dust_detection = np.ma.masked_where(dqf & 64 == 64, dust_detection)
    # Select smoke & dust pixels within valid SZA/VZA using "DQF" bit 7
    # Flag values: within valid SZA/VZA = 0, outside of valid SZA/VZA = 128
    smoke_detection = np.ma.masked_where(dqf & 128 == 128, smoke_detection)
    dust_detection = np.ma.masked_where(dqf & 128 == 128, dust_detection)

    return smoke_detection, dust_detection

In [5]:
# Calculate latitude & longitude from GOES ABI fixed grid projection (using xarray)
def calculate_abi_lat_lon(ds):
    
    # Read in GOES ABI fixed grid projection variables and constants
    x_coordinate_1d = ds.x  # E/W scanning angle in radians
    y_coordinate_1d = ds.y  # N/S elevation angle in radians
    projection_info = ds.goes_imager_projection  # ABI fixed grid contstants
    lon_origin = projection_info.longitude_of_projection_origin
    H = projection_info.perspective_point_height+projection_info.semi_major_axis
    r_eq = projection_info.semi_major_axis
    r_pol = projection_info.semi_minor_axis
    
    # Create 2D coordinate matrices from 1D coordinate vectors
    x_coordinate_2d, y_coordinate_2d = np.meshgrid(x_coordinate_1d, y_coordinate_1d)
    
    # Equations to calculate latitude and longitude
    lambda_0 = (lon_origin*np.pi)/180.0  
    a_var = np.power(np.sin(x_coordinate_2d),2.0) + (np.power(np.cos(x_coordinate_2d),2.0)*(np.power(np.cos(y_coordinate_2d),2.0)+(((r_eq*r_eq)/(r_pol*r_pol))*np.power(np.sin(y_coordinate_2d),2.0))))
    b_var = -2.0*H*np.cos(x_coordinate_2d)*np.cos(y_coordinate_2d)
    c_var = (H**2.0)-(r_eq**2.0)
    r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)
    s_x = r_s*np.cos(x_coordinate_2d)*np.cos(y_coordinate_2d)
    s_y = - r_s*np.sin(x_coordinate_2d)
    s_z = r_s*np.cos(x_coordinate_2d)*np.sin(y_coordinate_2d)
    
    lat = (180.0/np.pi)*(np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))
    lon = (lambda_0 - np.arctan(s_y/(H-s_x)))*(180.0/np.pi)

    lon = np.nan_to_num(lon, nan = -999.99)
    lat = np.nan_to_num(lat, nan = -999.99)
    return lat, lon

In [6]:
year, julian, hour = current_date()

fs = s3fs.S3FileSystem(anon=True)

# noaa-goes17, noaa-goes18
bucket = 'noaa-goes16'  

2023 294 13


In [7]:
# Products
products = {'adpc': 'ABI-L2-ADPC', 'aodc': 'ABI-L2-AODC', 'fdcc': 'ABI-L2-FDCC', 'glm': 'GLM-L2-LCFA'}

# adpc
# data_path_prev = bucket + '/' + product + '/'  + year + '/' + julian + '/' + prev_hour
adpc_data = bucket + '/' + products['adpc'] + '/'  + year + '/' + julian + '/' + hour 
adpc_files = fs.ls(adpc_data)
print(adpc_files[0])
adpc_ds = open_file_xarray(adpc_files[0])

# Calculate latitude and longitude using function defined previously
lat, lon = calculate_abi_lat_lon(adpc_ds)

# Process ADP smoke and dust detection using function defined previously
smoke_detection, dust_detection = process_abi_adp_detection(adpc_ds)


# fdcc
fdcc_data = bucket + '/' + products['fdcc'] + '/'  + year + '/' + julian + '/' + hour 
fdcc_files = fs.ls(fdcc_data)
print(fdcc_files[0])
fdcc_ds = open_file_xarray(fdcc_files[0])
frp = fdcc_ds.Power

# glm
glm_data = bucket + '/' + products['glm'] + '/'  + year + '/' + julian + '/' + hour 
glm_files = fs.ls(glm_data)
print(glm_files[0])
glm_ds = open_file_xarray(glm_files[0])

# Create arrays of the flash lat and lons in order to plot flash points
flash_lat = glm_ds.variables['flash_lat'][:]
flash_lon = glm_ds.variables['flash_lon'][:]
# Create arrays of the flash area and energy, and get the time variable from the .nc file attributes
flash_area = glm_ds.variables['flash_area'][:]
flash_energy = glm_ds.variables['flash_energy'][:]

noaa-goes16/ABI-L2-ADPC/2023/294/13/OR_ABI-L2-ADPC-M6_G16_s20232941301173_e20232941303546_c20232941304525.nc


/tmp/ipykernel_378/2297601066.py:21: RuntimeWarning: invalid value encountered in sqrt
  r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)


noaa-goes16/ABI-L2-FDCC/2023/294/13/OR_ABI-L2-FDCC-M6_G16_s20232941301173_e20232941303546_c20232941304124.nc
noaa-goes16/GLM-L2-LCFA/2023/294/13/OR_GLM-L2-LCFA_G16_s20232941300000_e20232941300200_c20232941300213.nc


In [8]:
# projection

projection_variables = adpc_ds.goes_imager_projection
satellite_height = projection_variables.perspective_point_height
semi_major_axis = projection_variables.semi_major_axis
semi_minor_axis = projection_variables.semi_minor_axis
central_longitude = projection_variables.longitude_of_projection_origin

globe = ccrs.Globe(semimajor_axis = semi_major_axis, semiminor_axis = semi_minor_axis)
geo_projection = ccrs.Geostationary(central_longitude = central_longitude, satellite_height = satellite_height,
                                    globe = globe, sweep_axis = 'x')

In [ ]:
# Set up figure in matplotlib
fig = plt.figure(figsize=(10, 10), dpi = 200)

# Set map projection using native geostationary map projection defined previously
ax = plt.axes(projection=geo_projection)

# Format lat/lon gridlines & labels using cartopy
gl = ax.gridlines(draw_labels=True, linewidth=0.25, color='silver', zorder=3)
gl.right_labels = None
gl.top_labels = None
gl.ypadding = 5
gl.xpadding = 5
gl.xlabel_style = {'size': 8}
gl.ylabel_style = {'size': 8}
gl.xformatter = LongitudeFormatter()
gl.yformatter = LatitudeFormatter()

# Add coastlines & borders (1:50m medium resolution)
# "zorder" argument sets order for plotting layers (larger zorder plots over smaller zorder); default zorder=1.5 
ax.add_feature(cfeature.NaturalEarthFeature(category='cultural', 
                                            name='admin_0_countries', 
                                            scale='50m'), 
               linewidth=0.75, facecolor='none', edgecolor='k', zorder=3)
ax.add_feature(cfeature.NaturalEarthFeature(category='cultural', 
                                            name='admin_1_states_provinces', 
                                            scale='50m'), 
               linewidth=0.5, facecolor='none', edgecolor='k', zorder=3)
ax.add_feature(cfeature.NaturalEarthFeature(category='physical', 
                                            name='ocean', 
                                            scale='50m'), 
               facecolor='cyan')
ax.add_feature(cfeature.NaturalEarthFeature(category='physical', 
                                            name='land', 
                                            scale='50m'), 
               facecolor='lightgreen')


box_array = [-120,-70,25,55]
ax.set_extent(box_array, ccrs.PlateCarree())

# Set colormaps for smoke and dust
cmap1 = mpl.colors.ListedColormap(['deeppink'])
cmap2 = mpl.colors.ListedColormap(['yellow'])
cmap3 = mpl.colors.ListedColormap(['orange'])
cmap4 = mpl.colors.ListedColormap(['grey'])

# Plot ADP smoke and dust detection
plot1 = ax.pcolormesh(lon, lat, smoke_detection, cmap = cmap1, transform = ccrs.PlateCarree(), zorder = 2)
plot2 = ax.pcolormesh(lon, lat, dust_detection, cmap = cmap2, transform = ccrs.PlateCarree(), zorder = 2)
plot3 = ax.pcolormesh(lon, lat, frp, cmap = cmap3)
plot4 = ax.hexbin(flash_lon, flash_lat, gridsize = 500, bins='log',transform = ccrs.PlateCarree(), cmap = cmap4, vmin = 1, vmax = 30, zorder = 2)

# Add smoke detection colorbar
cb1_ax = ax.inset_axes([0.05, -0.1, 0.15, 0.03])
cb1 = mpl.colorbar.ColorbarBase(cb1_ax, cmap=cmap1, orientation='horizontal')
cb1.set_label(label='Smoke', size=8, weight='bold')
cb1.ax.xaxis.set_major_formatter(ticker.NullFormatter())
cb1.ax.tick_params(which='major', length=0)

# Add dust detection colorbar
cb2_ax = ax.inset_axes([0.3, -0.1, 0.15, 0.03])
cb2 = mpl.colorbar.ColorbarBase(cb2_ax, cmap=cmap2, orientation='horizontal')
cb2.set_label(label='Dust', size=8, weight='bold')
cb2.ax.xaxis.set_major_formatter(ticker.NullFormatter())
cb2.ax.tick_params(which='major', length=0)

# fire
cb3_ax = ax.inset_axes([0.55, -0.1, 0.15, 0.03])
cb3 = mpl.colorbar.ColorbarBase(cb3_ax, cmap=cmap3, orientation='horizontal')
cb3.set_label(label='Fire', size=8, weight='bold')
cb3.ax.xaxis.set_major_formatter(ticker.NullFormatter())
cb3.ax.tick_params(which='major', length=0)

# lightning
cb4_ax = ax.inset_axes([0.8, -0.1, 0.15, 0.03])
cb4 = mpl.colorbar.ColorbarBase(cb4_ax, cmap=cmap4, orientation='horizontal')
cb4.set_label(label='Lightning', size=8, weight='bold')
cb4.ax.xaxis.set_major_formatter(ticker.NullFormatter())
cb4.ax.tick_params(which='major', length=0)

plt.show()